# Extract results of activation computations for UPP08 to RWCL

Dmitry Portnov, May 2021

## Goals and input data

Extract data from "component_to_cell.json" and form a table of values corresponding to RWCL "Radwaste_Checklist_UPP08.xlsx"
The both files are in the same folder, on 10.106.203.11 this is "d:\dvp\dev\mcnp\upp\wrk\up08".

## Scenario

- Load data from json as map RWCL item -> list of cells in the MCNP model
- Load activation data as map cell -> activation values for 12 day of cooliing



In [1]:
%config Completer.use_jedi = False

In [35]:
import typing as tp
from typing import Dict

In [2]:
import os, sys

In [3]:
try:
    import matplotlib as mpl
except ImportError:
    !pip install matplotlib
    import matplotlib as mpl

import matplotlib.pyplot as plt 
%matplotlib inline

In [4]:
import json
import pathlib

from collections import defaultdict
from pathlib import Path
from pprint import pprint, pformat


In [5]:
import numpy as np
import dotenv
from tqdm import tqdm

In [6]:
try:
    import pandas as pd
except ImportError:
    !pip install pandas
    import pandas as pd

In [7]:
try:
    import openpyxl
except ImportError:
    !pip install openpyxl
    import openpyxl

In [8]:
import mckit as mc
import r2s_rfda as r2s
import r2s_rfda.fetch as r2sft
import r2s_rfda.utils as r2sut
import r2s_rfda.launcher as r2sln

# from r2s_rfda.fetch import load_result_config, load_data, load_result_config
# from r2s_rfda import utils
# from r2s_rfda.launcher import load_config


---*** D:\programs\anaconda3\envs\r2s-0.3\lib\site-packages\mckit\nlopt.dll


In [9]:
# If exists local ".env" file, then load it to environment.
dotenv.load_dotenv();

In [10]:
def all_exist(*paths: Path) -> bool:
    return all(map(Path.exists, paths))
def all_are_files(*paths: Path) -> bool:
    return all(map(Path.is_file, paths))
def all_are_dirs(*paths: Path) -> bool:
    return all(map(Path.is_dir, paths))


In [102]:
ROOT: Path = Path(os.getenv("UP08_ACT_ROOT", r'd:\dvp\dev\mcnp\upp\wrk\up08'))
r2swrk: Path = ROOT / "r2s"
pjson: Path = ROOT / "component_to_cell-up08-1.json"
rwclinput: Path = ROOT / "Radwaste_Checklist_UPP08.xlsx"
rwclinput_iss: Path = ROOT / "Radwaste_Checklist_UP08_ISS_PCSS.xlsx"

assert all_exist(ROOT, r2swrk, pjson, rwclinput, rwclinput_iss)
assert all_are_dirs(ROOT, r2swrk)
assert all_are_files(pjson, rwclinput, rwclinput_iss)


## Load RWCL->cells mapping from JSON

In [13]:
with pjson.open() as fid:
    djson = json.load(fid)
pprint(sorted(djson.keys()))

['Bioshield plug',
 'Blade 1',
 'Bogies chasis left',
 'Bogies chasis right',
 'Bottom Frame',
 'Bottom Rail',
 'CASK docking connector right',
 'Cask docking connector left',
 'Cask docking rail left',
 'Cask docking rail structure right',
 'DFW Right',
 'DFW_left',
 'DMS SPI Reserve Tank',
 'DSM front',
 'DSM mid',
 'DSM rear',
 'DSM rear2',
 'DSM_mid',
 'Flange 1',
 'Flange ext',
 'HA interspace',
 'HA1',
 'HA2',
 'ISS chasis',
 'Interspace frame',
 'Interspace inner block',
 'Interspace left block',
 'Interspace right block',
 'Interspace zone',
 'Mount bottom',
 'Mount left',
 'Mount right',
 'Mount up',
 'Plate left 2',
 'Rail ISS left',
 'Rail ISS right',
 'Rear Box',
 'Rear left',
 'Rear right',
 'Shielding segment 1',
 'Shielding segment 2',
 'Shielding segment 3',
 'Shielding segment 4',
 'Shielding segment 5',
 'Shielding segment 6',
 'Shielding segment 7',
 'Shielding segment 8',
 'Support frame',
 'Top Frame',
 'Top Rail',
 'Trapezoid',
 'Tube left down',
 'Tube left up',


## Load Cell->activation data mapping from R2S working folder

In [15]:
r2s_launch_conf = r2sln.load_config(r2swrk)
r2s_result_conf = r2sft.load_result_config(r2swrk)

In [16]:
time = r2sut.convert_time_literal('12d')
time_labels = list(sorted(r2s_result_conf['gamma'].keys()))
time += time_labels[r2s_launch_conf['zero']]
closest_lab = r2sut.find_closest(time, time_labels)
# closest_lab

In [17]:
r2s_result_conf.keys()

dict_keys(['gamma', 'atoms', 'activity'])

In [18]:
activation_paths = r2s_result_conf["activity"]
activation_paths
act_data = r2sft.load_data(r2swrk / activation_paths[closest_lab])

In [19]:
# Estimate correction factor for tqdm progressbar total value.
its=80625951000
its/act_data._data.nnz

6349.120992777404

In [20]:
tqdm_correction = 6400

In [25]:
def load_cell_activity(cell_excel_path: Path):
    if cell_excel_path.exists():
        cell_activity_df = pd.read_excel(cell_excel_path, sheet_name="activity", index_col=0)
    else:
        cell_activity = defaultdict(float)
        with tqdm(total=act_data._data.nnz * tqdm_correction) as pbar:
            for cnt, ((g, c, i, j, k), act) in enumerate(act_data.iter_nonzero()):
                cell_activity[c] += act
                #print(act_data.gbins[g], c, i, j, k, act)
                #i += 1
                #if i == 1000:
                #    break
                if cnt % 1000 == 0:
                    pbar.update(cnt)
        cell_excel_path.parent.mkdir(parents=True, exist_ok=True)
        index = np.array(sorted(cell_activity.keys()), dtype=np.int32)
        data  = np.fromiter((cell_activity[k] for k in index), dtype=float)
        cell_activity_df = pd.DataFrame(data, columns=["activity"], index=index)
        with pd.ExcelWriter(cell_excel_path, engine="openpyxl") as excel:
            cell_activity_df.to_excel(excel, sheet_name="activity")
    return cell_activity_df
    

In [26]:
cell_activity_df = load_cell_activity(Path.cwd().parent.parent / "wrk/up08/cell_data.xlsx")
cell_activity_df

,activity
320000,4.721641e+13
320001,1.122701e+08
320002,9.906696e+08
320003,4.573004e+07
320004,9.968566e+08
...,...
323774,7.584694e+05
323775,6.154785e+05
323776,5.957511e+06
323777,5.307759e+06


In [27]:
cell_activity_df.loc[320000]["activity"]

47216410867303.66

In [47]:
rwclinput

WindowsPath('d:/dvp/dev/mcnp/upp/wrk/up08/Radwaste_Checklist_UPP08.xlsx')

In [96]:
rwclinput_df = pd.read_excel(rwclinput, sheet_name="UPP02", header=0, usecols=[2,3], index_col=0).iloc[3:]  # Yes, UPP02 for UPP08
rwclinput_df

,Mass of component (see 4.3)
Equipment part (see 4.1),
DFW Left,1048 kg
DFW Right,1004 kg
DSM,3629 kg
Side plate #1-18 (18 pieces),27 kg each
Frame Top,63 kg
Frame Bottom,138 kg
Frame Blade #1,21 kg
Frame Blade #2,21 kg
Frame Blade #3,19 kg


In [104]:
rwclinput_iss_df = pd.read_excel(rwclinput_iss, sheet_name="UPP02", header=0, usecols=[2,3], index_col=0).iloc[3:]  # Yes, UPP02 for UPP08
rwclinput_iss_df

,Mass of component (see 4.3)
Equipment part (see 4.1),
PC Locking System,118.0
PC Bogies,1812.0
PC Frame,NaN
PC Chasis,1224.0
IS Diagnostic sheet,441.0
IS Frame,245.0
IS Chasis,4800.0
IS Bogies,2420.0
IS Locking System,82.0


In [97]:
rwclinput_df.index

Index(['DFW Left', 'DFW Right', 'DSM', 'Side plate #1-18 (18 pieces)',
       'Frame Top', 'Frame Bottom', 'Frame Blade #1', 'Frame Blade #2',
       'Frame Blade #3', 'Frame Blade #4', 'Frame Blade #5', 'Frame Blade #6',
       'Frame Blade #7', 'Frame Blade #8', 'Frame Blade #9',
       'DSM Rails (2 pieces)', 'PP Tubes',
       'PP Generics (pads, skids, gripping - 8 pieces)', 'PP structure',
       'Generic Screws (50 pieces)', 'Rear frame shielding1',
       'Rear frame shielding2', 'Rear frame shielding3',
       'Rear frame shielding4', 'Rear frame shielding5',
       'Rear frame shielding6', 'Rear frame shielding7',
       'Rear frame shielding8', 'Rear frame shielding9',
       'Rear box shielding (left)', 'Rear box shielding (right)',
       'Rear box shielding plates (8 pieces)', 'Manifold'],
      dtype='object', name='Equipment part (see 4.1)')

In [105]:
inp1 = set(rwclinput_df.index.values)
inp2 = set(rwclinput_iss_df.index.values)
available = set(djson.keys())

In [106]:
sorted(inp1), sorted(inp2), sorted(available)

(['DFW Left',
  'DFW Right',
  'DSM',
  'DSM Rails (2 pieces)',
  'Frame Blade #1',
  'Frame Blade #2',
  'Frame Blade #3',
  'Frame Blade #4',
  'Frame Blade #5',
  'Frame Blade #6',
  'Frame Blade #7',
  'Frame Blade #8',
  'Frame Blade #9',
  'Frame Bottom',
  'Frame Top',
  'Generic Screws (50 pieces)',
  'Manifold',
  'PP Generics (pads, skids, gripping - 8 pieces)',
  'PP Tubes',
  'PP structure',
  'Rear box shielding (left)',
  'Rear box shielding (right)',
  'Rear box shielding plates (8 pieces)',
  'Rear frame shielding1',
  'Rear frame shielding2',
  'Rear frame shielding3',
  'Rear frame shielding4',
  'Rear frame shielding5',
  'Rear frame shielding6',
  'Rear frame shielding7',
  'Rear frame shielding8',
  'Rear frame shielding9',
  'Side plate #1-18 (18 pieces)'],
 ['Bioshield 1',
  'Bioshield 10',
  'Bioshield 11',
  'Bioshield 12',
  'Bioshield 13',
  'Bioshield 14',
  'Bioshield 15',
  'Bioshield 16',
  'Bioshield 17',
  'Bioshield 18',
  'Bioshield 19',
  'Bioshield 

In [112]:
inp1.intersection(available)

{'DFW Right'}

In [114]:
inp2.intersection(available)

set()

In [42]:
def calc_rwcl_item_activity(cell_activity_df: pd.DataFrame, djson: Dict[str, int]) -> tp.Tuple[Dict[str, float], Dict[str, tp.List[int]]]:
    missed_items_cells = defaultdict(list)
    rwcl_item_activity = defaultdict(float)
    for i, (k, v) in enumerate(djson.items()):
        for c in v:
            if c in cell_activity_df:
                rwcl_item_activity[k] += cell_activity_df.loc[c]
            else:
                missed_items_cells[k].append(c)
    return rwcl_item_activity, missed_items_cells

In [43]:
rwcl_item_activity, missed_items_cells = calc_rwcl_item_activity(cell_activity_df, djson)

In [44]:
rwcl_item_activity

defaultdict(float, {})

## Map RWCL items to activity value

In [46]:
# missed_items_cells

In [185]:
# sorted(rwcl_item_activity.keys())

In [ ]:
# 6                       Side plate #1-18 (18 pieces)
# 17                                    Frame Blade #9
# 18                              DSM Rails (2 pieces)
# 19                                          PP Tubes
# 20    PP Generics (pads, skids, gripping - 8 pieces)
# 22                        Generic Screws (50 pieces)
# 34              Rear box shielding plates (8 pieces)
# 35                                          Manifold
# Name: Equipment part (see 4.1), dtype: object
        
rwcl2json = {
 "Frame Blade #1": 'Frame1',
 "Frame Blade #2": 'Blade2',
 "Frame Blade #3": 'Blade3',
 "Frame Blade #4": 'Blade4',
 "Frame Blade #5": 'Blade5',
 "Frame Blade #6": 'Blade6',
 "Frame Blade #7": 'Blade7',
 "Frame Blade #8": 'Blade8',
 "DFW Left": 'DFW_left',
 "DFW Right": 'DFW_right',
#  'DMS_front',
#  'DMS_rear',
 "DSM": ['DSM','DSM_down', 'DSM_up'],
    
 "PP structure": [
     'Flange',
     'Flange_ext',
     'Trapezoid',
     'Rear_box',
 ]

 "Frame Bottom": 'Frame_bottom',
 "Frame Top": 'Frame_top',
# 'GDC',
 'Interspace_left_block',
 'Interspace_left_frame',
 'Interspace_lower_shielding',
 'Interspace_right_block',
 'Interspace_right_frame',
 'Interspace_top_frame',
 'Mount_bottom',
 'Mount_left',
 'Mount_right',
 'Mount_up',
#  'Reflectometer_left',
#  'Reflectometer_rear',
#  'Reflectometer_right',
 "Rear frame shielding1": 'Shielding1',
#  "Rear frame shielding10": ,
 "Rear frame shielding2": 'Shielding2',
 "Rear frame shielding3": 'Shielding3',
 "Rear frame shielding4": 'Shielding4',
 "Rear frame shielding5": 'Shielding5',
 "Rear frame shielding6": 'Shielding6',
 "Rear frame shielding7": 'Shielding7',
 "Rear frame shielding8": 'Shielding8',
 "Rear frame shielding8": 'Shielding_segment_9',
 "Rear box shielding (left)": 'Shielding_rear_left',
 "Rear box shielding (rightt)": 'Shielding_rear_right',
#  'Vis_exvessel',
#  'Visir_flange',
#  'Visir_invessel',
]

In [186]:
activation_column = np.zeros(len(rwclout_keys), dtype=float)

for i, key in enumerate(rwclout_keys):
    if key is np.NaN:
        activation_column[i] = np.NaN
    else:
        activity = rwcl_item_activity.get(key, None)
        if activity is None:
            print(f"No data for component {i} {key}")
            activation_column[i] = np.NaN
        else:
            activation_column[i] = activity
    
activation_column

No data for component 3 DFW Left
No data for component 4 DFW Right
No data for component 6 Side plate #1-18 (18 pieces)
No data for component 7 Frame Top
No data for component 8 Frame Bottom
No data for component 9 Frame Blade #1
No data for component 10 Frame Blade #2
No data for component 11 Frame Blade #3
No data for component 12 Frame Blade #4
No data for component 13 Frame Blade #5
No data for component 14 Frame Blade #6
No data for component 15 Frame Blade #7
No data for component 16 Frame Blade #8
No data for component 17 Frame Blade #9
No data for component 18 DSM Rails (2 pieces)
No data for component 19 PP Tubes
No data for component 20 PP Generics (pads, skids, gripping - 8 pieces)
No data for component 21 PP structure
No data for component 22 Generic Screws (50 pieces)
No data for component 23 Rear frame shielding1
No data for component 24 Rear frame shielding2
No data for component 25 Rear frame shielding3
No data for component 26 Rear frame shielding4
No data for componen

array([           nan,            nan,            nan,            nan,
                  nan, 9.57321799e+13,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan])

In [170]:
rwclinput_df = pd.read_excel(rwclinput, sheet_name="UPP02", engine="openpyxl")   # Yes, sheet name is "UPP02" in file for UP08 RWCL

In [175]:
rwclout_keys = rwclinput_df[rwclinput_df.columns[2]]

In [176]:
rwclout_keys

0                                                NaN
1                                                NaN
2                                                NaN
3                                           DFW Left
4                                          DFW Right
5                                                DSM
6                       Side plate #1-18 (18 pieces)
7                                          Frame Top
8                                       Frame Bottom
9                                     Frame Blade #1
10                                    Frame Blade #2
11                                    Frame Blade #3
12                                    Frame Blade #4
13                                    Frame Blade #5
14                                    Frame Blade #6
15                                    Frame Blade #7
16                                    Frame Blade #8
17                                    Frame Blade #9
18                              DSM Rails (2 p